# Document Augmentation RAG with Question Generation

문서의 각 부분마다 관련된 질문을 만들어서, RAG(검색 증강 생성) 방식이 더 똑똑하게 작동하도록 도와줍니다. 


#### 이 노트북의 주요 단계는 다음과 같습니다:
1. **데이터 불러오기**: PDF 파일에서 텍스트를 추출합니다.
2. **청크 나누기**: 긴 텍스트를 다루기 쉬운 조각들로 나눕니다.
3. **질문 생성**: 각 조각마다 관련된 질문을 만듭니다.
4. **임베딩 생성**: 조각과 질문에 대한 임베딩을 만듭니다.
5. **벡터 저장소 만들기**: NumPy를 이용해 간단한 벡터 저장소를 만듭니다.
6. **의미 기반 검색**: 사용자의 질문에 맞는 조각과 질문을 찾아줍니다.
7. **답변 생성**: 찾은 내용을 바탕으로 답변을 만듭니다.

## 환경 설정하기
필요한 라이브러리를 가져옵니다.

In [ ]:
import os
import numpy as np
from dotenv import load_dotenv
import re
from tqdm import tqdm

# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('OPENAI_API_KEY')

# OpenAI API 클라이언트 설정하기

In [ ]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)

## PDF 파일에서 텍스트 추출하기
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 저는 gemini를 이용해 pdf에서 텍스트를 추출하는 방식을 사용합니다.  
만약 txt 형태로 파일이 존재한다면 `load_text_file` 함수를 사용하면됩니다.

In [ ]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text

In [ ]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

extracted_text = load_text_file(txt_path)
print(extracted_text[:500])

## 추출된 텍스트 청크 분할
텍스트를 추출한 뒤에는 검색 정확도를 높이기 위해 조금씩 겹치도록 나눠서 작은 단위로 분할(chunk)합니다.  
이번 노트북에서는 여러 사이즈로 chunk를 분할합니다.

In [ ]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n자 단위로, 일부가 겹치도록 chunking 합니다.

    Args:
    text (str): 청크할 텍스트입니다.
    n (int): 각 청크의 문자 수입니다.
    overlap (int): 청크 간 겹치는 문자 수입니다.

    Returns:
    List[str]: 청크된 텍스트 리스트입니다.
    """
    chunks = []  # 청크된 텍스트를 저장할 리스트
    
    # overlap만큼 겹치도록 text를 n의 길이로 chunking
    for i in range(0, len(text), n - overlap):
        chunks.append(text[i:i + n])

    return chunks  

## chunk에 대한 질문 생성

LLM을 이용해 각각의 청크를 활용해 적절한 질문을 생성합니다.

In [ ]:
def generate_questions(text_chunk, num_questions=5, model="gpt-4.1-mini"):
    """
    text chunk 에 대한 질문 생성
    
    Args:
    text_chunk (str): 질문을 생성할 텍스트 청크.
    num_questions (int): 생성할 질문의 수.
    model (str): 질문 생성에 사용할 모델.

    Returns:
    List[str]: List of generated questions.
    """
    # 시스템 프롬프트 
    system_prompt = "You are an expert at generating relevant questions from text. Create concise questions that can be answered using only the provided text. Focus on key information and concepts."
    
    # user 프롬프트
    user_prompt = f"""
    Based on the following text, generate {num_questions} different questions that can be answered using only this text:

    {text_chunk}
    
    Format your response as a numbered list of questions only, with no additional text.
    """
    
    # 응답 생성
    response = client_openai.chat.completions.create(
        model=model,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # 응답에서 질문 추출
    questions_text = response.choices[0].message.content.strip()
    questions = []
    
    # 정규식을 사용해 질문 추출
    for line in questions_text.split('\n'):
        # 번호 제거 및 공백 정리
        cleaned_line = re.sub(r'^\d+\.\s*', '', line.strip())
        if cleaned_line and cleaned_line.endswith('?'):
            questions.append(cleaned_line)
    
    return questions

## 임베딩 생성
text chunk와 생성된 질문에 대한 임베딩을 모두 생성합니다.

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 인코딩을 위한 배치 크기입니다.

    Returns:
        np.ndarray: 모델에 의해 생성된 임베딩입니다.
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위한 진행 표시줄을 표시합니다.
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'mps'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

## Simple Vector Store 구축
NumPy를 사용하여 간단한 Vecotr store 구축

In [ ]:
class SimpleVectorStore:
    """
    NumPy를 사용하여 간단한 Vecotr store 구축
    """
    def __init__(self):
        """
        벡터 저장소 초기화
        """
        self.vectors = []
        self.texts = []
        self.metadata = []
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 항목 추가

        Args:
        text (str): 원본 텍스트.
        embedding (List[float]): 임베딩 벡터.
        metadata (dict, optional): 추가 메타데이터.
        """
        self.vectors.append(np.array(embedding))
        self.texts.append(text)
        self.metadata.append(metadata or {})
    
    def similarity_search(self, query_embedding, k=5):
        """
        시맨틱 서치 수행

        Args:
        query_embedding (List[float]): 쿼리 임베딩 벡터.
        k (int): 반환할 결과의 수.

        Returns:
        List[Dict]: 텍스트와 메타데이터가 포함된 상위 k개 유사 항목.
        """
        if not self.vectors:
            return []
        
        # 쿼리 임베딩을 numpy 배열로 변환
        query_vector = np.array(query_embedding)
        
        # 코사인 유사도를 사용하여 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))
        
        # 유사도에 따라 정렬 (내림차순)
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개 결과 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score
            })
        
        return results

## Processing Documents with Question Augmentation
이제 모든 과정을 결합하여 문서 처리, chunk 별 질문 생성, 벡터 스토어 구축 진행

In [ ]:
def process_document(file_path, chunk_size=1000, chunk_overlap=200, questions_per_chunk=5):
    """
    질문 생성을 통한 Document Augmentation 

    Args:
    pdf_path (str): PDF 파일 경로.
    chunk_size (int): 각 텍스트 청크의 크기(문자).
    chunk_overlap (int): 청크 간 겹치는 문자 수.
    questions_per_chunk (int): 각 청크당 생성할 질문의 수.

    Returns:
    Tuple[List[str], SimpleVectorStore]: 텍스트 청크와 벡터 저장소.
    """
    # print("Extracting text from PDF...")
    # extracted_text = extract_text_from_pdf(file_path)

    # 텍스트 파일 로드
    extracted_text = load_text_file(file_path)
    
    print("Chunking text...")
    text_chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"Created {len(text_chunks)} text chunks")
    
    vector_store = SimpleVectorStore()
    
    print("Processing chunks and generating questions...")
    for i, chunk in enumerate(tqdm(text_chunks, desc="Processing Chunks")):
        
        # 청크 임베딩 생성        
        chunk_embedding = create_embeddings(embedding_model, [chunk], device=device, batch_size=1)
        
        # 벡터 저장소에 청크 추가 및 임베딩 추가
        vector_store.add_item(
            text=chunk,
            embedding=chunk_embedding[0],
            metadata={"type": "chunk", "index": i}
        )
        
        # 질문 생성
        questions = generate_questions(chunk, num_questions=questions_per_chunk)
        
        # 질문 임베딩 생성 및 벡터 저장소에 추가
        for j, question in enumerate(questions):
            
            question_embedding = create_embeddings(embedding_model, [question], device=device, batch_size=1)
            
            # 벡터 저장소에 질문 추가 및 임베딩 추가
            vector_store.add_item(
                text=question,
                embedding=question_embedding[0],
                metadata={"type": "question", "chunk_index": i, "original_chunk": chunk}
            )
    
    return text_chunks, vector_store

## Extracting and Processing the Document

In [ ]:
# 텍스트 파일 경로 설정
txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

# 문서 처리 (텍스트 추출, 청크 생성, 질문 생성, 벡터 저장소 구축)   
text_chunks, vector_store = process_document(
    txt_path, 
    chunk_size=1000, 
    chunk_overlap=200, 
    questions_per_chunk=3
)

print(f"Vector store contains {len(vector_store.texts)} items")

## Semantic Search 수행

 simple RAG 구현과 유사하게 벡터 저장소에 맞춘 Semantic Search 함수 구현

In [ ]:
def semantic_search(query, vector_store, k=5):
    """
    semantic serach 수행

    Args:
    query (str): 검색 쿼리.
    vector_store (SimpleVectorStore): 검색할 벡터 저장소.
    k (int): 반환할 결과의 수.

    Returns:
    List[Dict]: Top k most relevant items.
    """
    # 쿼리 임베딩 생성
    query_embedding = create_embeddings(embedding_model, [query], device=device, batch_size=1)
    query_embedding = query_embedding[0]
    
    # 벡터 저장소에서 관련 chunk 검색
    results = vector_store.similarity_search(query_embedding, k=k)
    
    return results

## 테스트

In [ ]:
import pandas as pd

# 평가 데이터 로드하기
df = pd.read_csv('./data_creation/rag_val_new_post.csv')
df.head()

In [ ]:
# 평가 데이터에서 첫 번째 쿼리 추출
query = df['query'][0]

# 시맨틱 search를 통해 관련 chunk 찾기
search_results = semantic_search(query, vector_store, k=3)

print("Query:", query)
print("\nSearch Results:")

# 결과 분류
chunk_results = []
question_results = []

for result in search_results:
    if result["metadata"]["type"] == "chunk":
        chunk_results.append(result)
    else:
        question_results.append(result)

# 청크 결과 출력
print("\nRelevant Document Chunks:")
for i, result in enumerate(chunk_results):
    print(f"Context {i + 1} (similarity: {result['similarity']:.4f}):")
    print(result["text"][:300] + "...")
    print("=====================================")

# 질문 결과 출력
print("\nMatched Questions:")
for i, result in enumerate(question_results):
    print(f"Question {i + 1} (similarity: {result['similarity']:.4f}):")
    print(result["text"])
    chunk_idx = result["metadata"]["chunk_index"]
    print(f"From chunk {chunk_idx}")
    print("=====================================")

In [ ]:
def prepare_context(search_results):
    """
    검색 결과를 기반으로 응답 생성을 위한 통합 컨텍스트 준비

    Args:
    search_results (List[Dict]): 시맨틱 검색 결과.

    Returns:
    str: 통합 컨텍스트 문자열.
    """
    # 결과에서 참조된 고유한 청크 추출
    chunk_indices = set()
    context_chunks = []
    
    # 직접 청크 매칭 추가
    for result in search_results:
        if result["metadata"]["type"] == "chunk":
            chunk_indices.add(result["metadata"]["index"])
            context_chunks.append(f"Chunk {result['metadata']['index']}:\n{result['text']}")
    
    # 질문에 의해 참조된 청크 추가
    for result in search_results:
        if result["metadata"]["type"] == "question":
            chunk_idx = result["metadata"]["chunk_index"]
            if chunk_idx not in chunk_indices:
                chunk_indices.add(chunk_idx)
                context_chunks.append(f"Chunk {chunk_idx} (referenced by question '{result['text']}'):\n{result['metadata']['original_chunk']}")
    
    # 모든 컨텍스트 청크 결합
    full_context = "\n\n".join(context_chunks)
    return full_context

## 검색된 청크를 기반으로 response 생성하기

In [ ]:
def generate_response(query, context, model_name='gpt-4.1-mini'):

    # 시스템 프롬프트
    system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"
    
    # 유저 프롬프트
    user_prompt = f"""
        Context:
        {context}

        Question: {query}

        Please answer the question based only on the context provided above. Be concise and accurate.
    """
    
    response = client_openai.chat.completions.create(
        model=model_name,
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return response.choices[0].message.content

## Generating and Displaying the Response

In [ ]:
# 컨텍스트 준비
context = prepare_context(search_results)

# 응답 생성
response_text = generate_response(query, context)
ai_response = generate_response(query, context,'gpt-4.1-nano-2025-04-14')
print("\nQuery:", query)
print("\nResponse:")
print(response_text)

## 생성 응답 평가하기
생성된 답변과 예상 답변을 비교하여 평가합니다. 답변 평가시에는 LLM을 사용합니다.

In [ ]:
def evaluate_response(query, response, reference_answer, model_name='gpt-4.1-mini'):
    """
    
    AI 응답을 기준 답변과 비교하여 평가합니다.
    
    Args:
    query (str): 사용자의 질문.
    response (str): AI 생성 응답.
    reference_answer (str): 기준/이상적 답변.
    model (str): 평가에 사용할 모델.
    
    Returns:
    str: 평가 피드백.
    """
    # 평가를 위한 시스템 프롬프트
    evaluate_system_prompt = """You are an intelligent evaluation system tasked with assessing AI responses.
            
        Compare the AI assistant's response to the true/reference answer, and evaluate based on:
        1. Factual correctness - Does the response contain accurate information?
        2. Completeness - Does it cover all important aspects from the reference?
        3. Relevance - Does it directly address the question?

        Assign a score from 0 to 1:
        - 1.0: Perfect match in content and meaning
        - 0.8: Very good, with minor omissions/differences
        - 0.6: Good, covers main points but misses some details
        - 0.4: Partial answer with significant omissions
        - 0.2: Minimal relevant information
        - 0.0: Incorrect or irrelevant

        Provide your score with justification.
    """
            
    # 평가 프롬프트
    evaluation_prompt = f"""
        User Query: {query}

        AI Response:
        {response}

        Reference Answer:
        {reference_answer}

        Please evaluate the AI response against the reference answer.
    """
    
    # 평가 응답 생성
    eval_response = client_openai.chat.completions.create(
        model=model_name,
        temperature=0,
        messages=[
            {"role": "system", "content": evaluate_system_prompt},
            {"role": "user", "content": evaluation_prompt}
        ]
    )
    
    return eval_response.choices[0].message.content

In [ ]:
# 참고(정답) 답변
reference_answer = df['generation_gt'][0]

# 응답 평가
evaluation = evaluate_response(query, response_text, reference_answer)

print("\nEvaluation:")
print(evaluation)